In [13]:
#https://www.idealista.pt/media/relatorios-preco-habitacao/venda/lisboa/historico/
#Historico de preços

#my user agent
"""Mozilla/5.0 (Windows NT 10.0; Win64; x64)
    AppleWebKit/537.36 (KHTML, like Gecko)
    Chrome/114.0.0.0 Safari/537.36"""

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

# **TODOS BAIRROS**

In [14]:
bairros_link = [
"https://supercasa.pt/comprar-casas/lisboa/santa-clara/",
"https://supercasa.pt/comprar-casas/lisboa/lumiar/",
"https://supercasa.pt/comprar-casas/lisboa/carnide/",
"https://supercasa.pt/comprar-casas/lisboa/benfica/",
"https://supercasa.pt/comprar-casas/lisboa/marvila/",
"https://supercasa.pt/comprar-casas/lisboa/alvalade/",
"https://supercasa.pt/comprar-casas/lisboa/sao-domingos-de-benfica/",
"https://supercasa.pt/comprar-casas/lisboa/areeiro",
"https://supercasa.pt/comprar-casas/lisboa/avenidas-novas/",
"https://supercasa.pt/comprar-casas/lisboa/campolide/",
"https://supercasa.pt/comprar-casas/lisboa/belem/",
"https://supercasa.pt/comprar-casas/lisboa/ajuda/",
"https://supercasa.pt/comprar-casas/lisboa/alcantara/",
"https://supercasa.pt/comprar-casas/lisboa/campo-de-ourique/",
"https://supercasa.pt/comprar-casas/lisboa/estrela/",
"https://supercasa.pt/comprar-casas/lisboa/santo-antonio/",
"https://supercasa.pt/comprar-casas/lisboa/misericordia/",
"https://supercasa.pt/comprar-casas/lisboa/arroios/",
"https://supercasa.pt/comprar-casas/lisboa/santa-maria-maior/",
"https://supercasa.pt/comprar-casas/lisboa/sao-vicente/",
"https://supercasa.pt/comprar-casas/lisboa/penha-de-franca/",
"https://supercasa.pt/comprar-casas/lisboa/beato/",
"https://supercasa.pt/comprar-casas/lisboa/parque-das-nacoes/",
"https://supercasa.pt/comprar-casas/lisboa/olivais/"]


bairros_link[23]

'https://supercasa.pt/comprar-casas/lisboa/olivais/'

In [15]:
# "BAIRROS NAMES"

bairros = ["santa-clara", "lumiar", "carnide", "benfica", "marvila", "alvalade", "sao-domingos-de-benfica", "areeiro", "avenidas-novas", "campolide", "belem", "ajuda", "alcantara", "campo-de-ourique", "estrela", "santo-antonio", "misericordia", "arroios", "santa-maria-maior", "sao-vicente", "penha-de-franca", "beato", "parque-das-nacoes", "olivais"]
len(bairros)


24

In [16]:
def get_houses_all (pages, n):
  new_bairro = {}
  count = 0
  #######################################
  url = bairros_link[n]
  response = requests.get(url)
  response

  soup = BeautifulSoup(response.content)

  homes = soup.find_all("div", attrs ={"class":"property big-picture"})
  homes[0].get_text().strip()


  for page in range(1,pages+1):  #we need to change the url respecting users input
      url = f"{bairros_link[n]}pagina-{page}"
      r = requests.get(url)
      soup = BeautifulSoup(r.content)

      books = soup.find_all("li", attrs = {"class":"col-xs-6 col-sm-4 col-md-3 col-lg-3"})

      for i in homes: #find intern func
          price = i.find_all("div", attrs={"class": "property-price"})[0].get_text().strip()
          size_text = i.find_all("div", attrs={"class": "property-features"})[0].get_text().strip()
          zone = i.find_all("h2", attrs={"class": "property-list-title"})[0].get_text().strip()


          num_rooms_match = re.search(r'(\d+)\s+quartos', size_text)
          num_rooms = int(num_rooms_match.group(1)) if num_rooms_match else None

          area_match = re.search(r'Área bruta (\d+)\s+m²', size_text)
          area = int(area_match.group(1)) if area_match else None

          terrain_match = re.search(r'Terreno (\d+)\s+m²' , size_text)
          terrain = int(terrain_match.group(1)) if terrain_match else None


          new_bairro[count] = {"Price": price,
                                "Num Rooms": num_rooms,
                                "Area (m²)": area,
                                "Terrain (m²)": terrain,
                                "Zone": zone}
          count += 1



  return pd.DataFrame.from_dict(new_bairro, orient="index")

In [17]:
#RUN ALL NEIG IN PAGE

bairro_dic = {}
for x in range(0,len(bairros)):
  new_bairro = get_houses_all(1,x)
  bairro_dic[bairros[x]] = new_bairro
  bairro_dic[bairros[x]]["Area (m²)"]=pd.to_numeric(new_bairro['Area (m²)'], errors="coerce") ###
  bairro_dic[bairros[x]]["Price"] = new_bairro['Price'].apply(lambda x: int(re.sub(r'\D', '', x)))
  bairro_dic[bairros[x]]["value_m2"] = round(new_bairro["Price"]/new_bairro["Area (m²)"],2)
  bairro_dic[bairros[x]]["freguesia"] = bairros[x]

In [18]:
lista_frames = list(bairro_dic.values())
result = pd.concat(lista_frames)
result

,Price,Num Rooms,Area (m²),Terrain (m²),Zone,value_m2,freguesia
0,234000,3.0,135.0,NaN,"Apartamento T3 na Rua Barata Feyo, Santa Clara...",1733.33,santa-clara
1,415000,3.0,116.0,NaN,"Apartamento T3 em Santa Clara, Lisboa",3577.59,santa-clara
2,287000,NaN,69.0,NaN,"Apartamento T1 em Santa Clara, Lisboa",4159.42,santa-clara
3,350000,2.0,106.0,NaN,Apartamento T2 na Avenida Sérgio Vieira de Mel...,3301.89,santa-clara
4,620000,3.0,147.0,NaN,"Apartamento T3 em Calçada do Poço, Santa Clara...",4217.69,santa-clara
...,...,...,...,...,...,...,...
6,600000,2.0,109.0,NaN,Apartamento T2 em Praça da Viscondessa dos Oli...,5504.59,olivais
7,1200000,NaN,NaN,201.0,"Moradia em Olivais, Lisboa",NaN,olivais
8,550000,3.0,85.0,234.0,"Moradia T3 Duplex em Alameda da Encarnação, Ol...",6470.59,olivais
9,1150000,3.0,199.0,NaN,Apartamento T3 em Praça da Viscondessa dos Oli...,5778.89,olivais


In [19]:
result.to_csv("ola.csv")

In [20]:
result.to_excel("geral.xlsx")